In [ ]:


import numpy as np 
import pandas as pd 


import openslide
import os
import cv2
import PIL
from PIL import Image
from IPython.display import Image, display
from tensorflow.python.keras.applications.vgg16 import VGG16,preprocess_input
import plotly.graph_objs as go
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model,load_model
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten,BatchNormalization,Activation
from keras.layers import GlobalMaxPooling2D
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense,Dropout,Activation,Flatten,Lambda
import gc
import matplotlib.pyplot as plt
%matplotlib inline
import skimage.io
from sklearn.model_selection import KFold
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3, VGG19, ResNet50
from tensorflow.keras.applications.inception_v3 import preprocess_input as process_inception_v3
from tensorflow.keras.applications.vgg19 import preprocess_input as process_vgg19
from tensorflow.keras.applications.resnet50 import preprocess_input as process_resnet
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *

import cv2
import random
import itertools
import os

from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.python.keras import backend as K
sess = K.get_session()

In [ ]:
from keras.callbacks.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
from keras.applications.xception import Xception
import time
import seaborn as sns
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [ ]:

from numpy.random import seed
seed(1)

In [ ]:
train=pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/train.csv')
train.head()
train_copy = train.copy()

In [ ]:
img=openslide.OpenSlide('/kaggle/input/prostate-cancer-grade-assessment/train_images/2fd1c7dc4a0f3a546a59717d8e9d28c3.tiff')
display(img.get_thumbnail(size=(512,512)))

In [ ]:
img.dimensions

In [ ]:
patch = img.read_region((18500,4100), 0, (256, 256))

# Display the image
display(patch)
# Close the opened slide after use
img.close()

In [ ]:
train['isup_grade'].value_counts()

The classes are imbalanced, more severe cases are underrepresented.

Let's start preparing the images for training.

In [ ]:
train.head()

In [ ]:
labels=[]
data=[]
data_dir='../input/panda-resized-train-data-512x512/train_images/train_images/'
for i in range(train.shape[0]):
    data.append(data_dir + train['image_id'].iloc[i]+'.png')
    labels.append(train['isup_grade'].iloc[i])
df=pd.DataFrame(data)
df.columns=['images']
df['isup_grade']=labels

In [ ]:
df.head()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df['images'],df['isup_grade'], test_size=0.1, random_state=42)


In [ ]:
train=pd.DataFrame(X_train)
train.columns=['images']
train['isup_grade']=y_train

validation=pd.DataFrame(X_val)
validation.columns=['images']
validation['isup_grade']=y_val

train['isup_grade']=train['isup_grade'].astype(str)
validation['isup_grade']=validation['isup_grade'].astype(str)

So the basic preprocessing I've done is:-
* Normalizing the images.
* Reshape the images to be of shape 224,224,3
* Basic image augmentation like rotation, flipping etc.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=40,
    featurewise_center=True,
    featurewise_std_normalization=True,
    #zoom_range=[0.8, 1.2],        
    horizontal_flip=True, vertical_flip = True,
    brightness_range=[0.9, 1.1],
    width_shift_range=1.0,
    height_shift_range=1.0)#,
    #validation_split=0.1)


val_datagen=train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col='images',
    y_col='isup_grade',
    target_size=(224, 224),
    batch_size=32,
    shuffle = True,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_dataframe(
    validation,
    x_col='images',
    y_col='isup_grade',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

In [ ]:
filenames = validation_generator.filenames
nb_samples = len(filenames)



In [ ]:
y_true = validation_generator.classes

# y_true

**VGG16**

In [ ]:
from tensorflow.python.keras.applications.vgg16 import VGG16
def vgg16_model(num_classes = None):
    vgg16_weights = '../input/vgg19/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

    model = VGG16(weights= '../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5' , include_top=False, input_shape=(224, 224, 3))    
    x=Dropout(0.3)(model.output)
    x=Flatten()(x)
    x =Dense(32, activation = 'relu')(x)
    x =Dropout(0.2)(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model
vgg_conv = vgg16_model(6)
vgg_conv.summary()

**VGG19**

In [ ]:
from tensorflow.python.keras.applications.vgg19 import VGG19
def vgg19_model(num_classes = None):
    vgg19_weights = '../input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'
    #model = VGG19(weights='imagenet',include_top=False, input_shape=(224, 224, 3))
    model = VGG19(weights= vgg19_weights , include_top=False, input_shape=(224, 224, 3))
    x=Dropout(0.3)(model.output)
    x=Flatten()(x)
    x =Dense(32, activation = 'relu')(x)
    x =Dropout(0.2)(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model
vgg19_conv = vgg19_model(6)
vgg19_conv.summary()

**InceptionV3 model**

In [ ]:
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
def InceptionV3_model(num_classes = None):
    InceptionV3_weights = '../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    model = InceptionV3(weights= InceptionV3_weights, include_top=False, input_shape=(224, 224, 3))
    #model = InceptionV3(weights='imagenet', include_top = False, input_shape = (224,224,3))
    x=Dropout(0.3)(model.output)
    #x=Flatten()(model.output)
    #x =Dense(64, activation = 'relu')(model.output)
    #x = tf.compat.v1.keras.layers.GlobalAveragePooling2D()(model.output)
    x=Flatten()(x)
    #x =Dropout(0.2)(x)
    x =Dense(32, activation = 'relu')(x)
    x =Dropout(0.2)(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model
InceptionV3_conv = InceptionV3_model(6)
InceptionV3_conv.summary()

**Resnet50 model**

In [ ]:

# def ResNet101_model(num_classes = None):
#     ResNet_weights = '../input/keras-pretrained-models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
#     #model = ResNet50(weights='imagenet', include_top = False, input_shape = (224,224,3))
#     model = tf.keras.applications.ResNet101(weights= 'imagenet', include_top=False, input_shape=(224, 224, 3))
#     #x=Dropout(0.2)(model.output)
#     #x = GlobalAveragePooling2D()(model.output)
#     x=Flatten()(model.output)
#     #x =Dropout(0.2)(x)
#     x =Dense(16, activation = 'relu')(x)
#     x =Dropout(0.2)(x)
#     output=Dense(num_classes,activation='softmax')(x)
#     model=Model(model.input,output)
#     return model
# ResNet50_conv = ResNet101_model(6)
# ResNet50_conv.summary()

In [ ]:
# def Resnet_model(num_classes = None):
# #     InceptionResnet_weights = '../input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
# #     model = inception_resnet_v2(weights='imagenet', include_top = False, input_shape = (224,224,3))
#     model = tf.keras.applications.ResNet50(include_top=False, input_shape=(224, 224, 3))
#     x=Dropout(0.2)(model.output)
#     x=Flatten()(model.output)
#     #x =Dropout(0.2)(x)
#     x =Dense(16, activation = 'relu')(x)
#     x =Dropout(0.2)(x)
#     output=Dense(num_classes,activation='softmax')(x)
#     model=Model(model.input,output)
#     return model
# Resnet50_conv = Resnet_model(6)
# Resnet50_conv.summary()


In [ ]:
from keras.applications import EfficientNetB0
model = EfficientNetB0(weights='imagenet')

**InceptionV3_conv,ResNet50_conv**

In [ ]:
!pip install keras-efficientnet
import efficientnet.tfkeras as efc
def Efficient_model(num_classes = None):
    model = efc(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape = (224,224,3),
    pooling=None,
    classes=6,
    classifier_activation="softmax",
)

#     model = tf.keras.applications.Efficenet(weights= '../input/keras-pretrained-models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224, 224, 3))
    x=Dropout(0.2)(model.output)
#     x = tf.compat.v1.keras.layers.GlobalAveragePooling2D()(model.output)
    x=Flatten()(model.output)
    #x =Dropout(0.2)(x)
    x =Dense(16, activation = 'relu')(x)
    x =Dropout(0.2)(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model
EfficientNet_conv = Efficient_model(6)
EfficientNet_conv.summary()

In [ ]:

def InceptionResnet_model(num_classes = None):
#     InceptionResnet_weights = '../input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
#     model = inception_resnet_v2(weights='imagenet', include_top = False, input_shape = (224,224,3))
    model = tf.keras.applications.InceptionResNetV2(weights= '../input/keras-pretrained-models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224, 224, 3))
    x=Dropout(0.2)(model.output)
#     x = tf.compat.v1.keras.layers.GlobalAveragePooling2D()(model.output)
    x=Flatten()(model.output)
    #x =Dropout(0.2)(x)
    x =Dense(16, activation = 'relu')(x)
    x =Dropout(0.2)(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model
InceptionResnet_conv = InceptionResnet_model(6)
InceptionResnet_conv.summary()

**Including batch Normalization**

In [ ]:
def freeze(model):
    # freeze layers before 99
    for layer in model.layers[:99]:
        layer.trainable = False
        if layer.name.startswith('batch_normalization'):
            layer.trainable = True
        if layer.name.endswith('bn'):
            layer.trainable = True

    # unfreeze layers after 99
    for layer in model.layers[99:]:
        layer.trainable = True
        
freeze(vgg_conv)
freeze(vgg19_conv)
freeze(InceptionV3_conv)
# freeze(ResNet50_conv)   
        

In [ ]:
#kappa score is the metric we use since two types of evaluation is done on the data set 

#The kappa statistic.
#According to Cohen's original article, 
#values ≤ 0 as indicating no agreement and 0.01–0.20 as none to slight,
#0.21–0.40 as fair, 0.41– 0.60 as moderate,
#0.61–0.80 as substantial, 
#0.81–1.00 as almost perfect agreement.

In [ ]:
def kappa_score(y_true, y_pred):
    
    y_true=tf.math.argmax(y_true)
    y_pred=tf.math.argmax(y_pred)
    return tf.compat.v1.py_func(cohen_kappa_score,(y_true, y_pred),tf.double)
    return tf.compat.v1.py_func(cohen_kappa_score,(y_true,y_pred),tf.double)
    return (y_true,y_pred)

**Hyperparameter Tuning**

In [ ]:
opt = Adam(lr= 0.0005)
vgg_conv.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
vgg19_conv.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
InceptionV3_conv.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
# ResNet50_conv.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
InceptionResnet_conv.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
nb_epochs = 50
batch_size=16
#nb_train_steps = train.shape[0]//batch_size
#nb_val_steps=validation.shape[0]//batch_size
nb_train_steps = 128
nb_val_steps = 64
print("Number of training and validation steps: {} and {}".format(nb_train_steps,nb_val_steps))

**VGG16 Training**

In [ ]:
vgg_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=50,validation_data=validation_generator,
validation_steps=nb_val_steps)


**VGG19 training**

In [ ]:
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=15, verbose=1, factor=0.5),
             EarlyStopping(monitor='val_loss', patience=15),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
vgg19_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=50,validation_data=validation_generator,validation_steps=nb_val_steps,callbacks = callbacks)

**inception training**

In [ ]:
history =InceptionV3_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=nb_epochs,validation_data=validation_generator,
validation_steps=nb_val_steps, callbacks = callbacks)

**Resnet50 training**

In [ ]:
# ResNet50_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=50,validation_data=validation_generator,
# validation_steps=nb_val_steps, callbacks = callbacks)

**inceptionResnet training**

In [ ]:
InceptionResnet_conv.fit_generator( train_generator,steps_per_epoch=nb_train_steps,epochs=50,validation_data=validation_generator,
validation_steps=nb_val_steps, callbacks = callbacks)

In [ ]:
inception_predictions = InceptionResnet_conv.predict(train_generator)

In [ ]:
vgg_predictions = vgg_conv.predict(train_generator)

In [ ]:
vgg_19_predictions= vgg19_conv.predict(train_generator)

In [ ]:
InceptionV3_predictions = InceptionV3_conv.predict(train_generator)

In [ ]:

incep_predictions = []
for i in range(0, inception_predictions.shape[0]):
    incep_predictions.append(np.argmax(inception_predictions[i]))
incep_pred = set(incep_predictions)
print(incep_pred )

In [ ]:
vgg_19_pred = []
for i in range(0, vgg_19_predictions.shape[0]):
    vgg_19_pred.append(np.argmax(vgg_19_predictions[i]))
vgg_19_pred = set(vgg_19_pred)
print(vgg_19_pred )

In [ ]:

vgg_pred = []
for i in range(0, vgg_predictions.shape[0]):
    vgg_pred.append(np.argmax(vgg_predictions[i]))
vgg_pred = set(vgg_pred)
print(vgg_pred )

In [ ]:

inceptionV3_pred = []
for i in range(0, InceptionV3_predictions.shape[0]):
    inceptionV3_pred.append(np.argmax(InceptionV3_predictions[i]))
inceptionV3_pred = set(inceptionV3_pred)
print(inceptionV3_pred)